In [ ]:
# prerequisite: 
# TODO

In [ ]:
# Here we ensure that the text sizes of our figures scale with the figures themselves.
# A change of figure width, which has to fit into column in latex document, affects perceived 
# text size; change of figure height does not affect perceived text size, provided the figure 
# height does not exceed the column height of the latex document

pt2inch = 72.27
w_onecol = 238.96417/pt2inch # numerator from \the\textwidth in relevant environment in latex document
w_twocol = 495.0/pt2inch     # numerator from \the\textwidth in relevant environment in latex document
wh_ratio = 6.4/4.8           # default width to height ratio in matplotlib

# height_scale can be changed with impunity. It allows for changing of the width to height 
#   ratio, without altering the perceived text sizes of the figures
# fig_scale can be changed with impunity. It simply changes the overall scale of the output 
#   figures, e.g. inside Jupyter Notebook and output .pdf files.
def prepare_one_column_figure(height_scale=1.0, fig_scale=1.0):
    plt.rc("pdf", fonttype=42)
    plt.rc("axes", labelsize=fig_scale*12)
    plt.rc("xtick", labelsize=fig_scale*12, top=True, direction="in")
    plt.rc("ytick", labelsize=fig_scale*12, right=True, direction="in")
    plt.rc("axes", titlesize=fig_scale*12)
    plt.rc("legend", fontsize=fig_scale*12, title_fontsize=fig_scale*12)
    plt.rc("font", size=fig_scale*12)
    return plt.figure(figsize=(fig_scale*w_onecol, fig_scale*height_scale*w_onecol/wh_ratio))

def prepare_two_column_figure(height_scale=1.0, fig_scale=1.0, nrows=1, ncols=1, 
                              sharex=False, sharey=False, gridspec_kw=None):
    plt.rc("pdf", fonttype=42)
    plt.rc("axes", labelsize=fig_scale*12)
    plt.rc("xtick", labelsize=fig_scale*12, top=True, direction="in")
    plt.rc("ytick", labelsize=fig_scale*12, right=True, direction="in")
    plt.rc("axes", titlesize=fig_scale*12)
    plt.rc("legend", fontsize=fig_scale*12, title_fontsize=fig_scale*12)
    plt.rc("font", size=fig_scale*12)
    if nrows == 1 and ncols == 1:
        return plt.figure(figsize=(fig_scale*w_twocol, fig_scale*height_scale*w_twocol/wh_ratio))
    else:
        return plt.subplots(nrows=nrows, ncols=ncols, 
                            figsize=(fig_scale*w_twocol, fig_scale*height_scale*w_twocol/wh_ratio), 
                            sharex=sharex, sharey=sharey, gridspec_kw=gridspec_kw)

In [ ]:
import uproot
import awkward as ak

import subprocess

import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

from scipy.optimize import curve_fit
from scipy.stats import norm
from scipy.stats import uniform

import os
curdir = os.getcwd()
while not os.path.exists(".projectroot") and os.getcwd() != os.path.expanduser('~'):
    os.chdir("..")
try:
    proot = open(".projectroot").readline()
except:
    print("Could not find file '.projectroot'.")
#     print("Fallback: Look for the file 'rpad.py' in the current working directory (%s)." % curdir)
    print("Fonts in figures might not appear as intended, either.")
os.chdir(curdir)
import sys
import matplotlib.font_manager as font_manager
if len(proot) > 0:
#     sys.path.append(proot+"/python")
    font_dir = proot + '/fonts'
    font_files = font_manager.findSystemFonts(font_dir)
    font_files = font_manager.findSystemFonts()
    #for f in font_files:
    #    font_manager.FontManager.addfont(font_manager.fontManager, path=f)
    #    print(f)
    plt.rcParams.update({"font.family": "serif", "font.serif": "Linux Libertine O", "font.cursive": "Linux Libertine O", "font.sans-serif": "Linux Libertine O", "mathtext.fontset": "custom"})
# from rpad import rpad

os.makedirs("tmp", exist_ok=True)

dssds = ["U3", "U4", "U5"]
ids = [1, 2, 3]

In [ ]:
# AME 2020. Masses in micro-u
mp  = 1007825.031898
mNa = 20997654.459
mNe = 19992440.17525
S1p = 5504.1000
S2p = 7935.9963
QB  = 18600

In [ ]:
def E2(E1, Q2, theta):
    E2p = Q2*mNe/(mNe+mp)
    return E2p + E1*(mp/mNa)**2 - 2*mp*np.sqrt(E1*E2p)*np.cos(theta)/mNa

def Q2p(E1, Q2):
    return E1*(mp+mNa)/mNa + Q2

In [ ]:
E1 = 1000
Q2 = 1500

q2p   = Q2p(E1, Q2)
e2min = E2(E1, Q2, 0)
e2max = E2(E1, Q2, np.pi)

#plt.plot([E1, e2min, e2max], q2p*np.ones(3), '.')

In [ ]:
E1     = np.linspace(0, 8.000, 100)
Q2s    = [0.05, 0.500, 2.000]
Q2strs = ["0.05", "0.5", "2.0"]
lss    = ['-', '--', '-.']
hatchs = ['-', 'x', 'o', '/', '\\', '|', '+', 'O', '.', '*']

thetamin = 0
thetamax = np.pi

ymax = Q2p(np.max(E1), np.max(Q2s))
for i in range(len(Q2s)):
    prepare_two_column_figure(fig_scale=1.5)
    for j in range(i+1):
        Q2 = Q2s[j]
        Q2str = Q2strs[j]
        ls = lss[j]
        hatch = hatchs[j]

        q2p   = Q2p(E1, Q2)
        e2min = E2(E1, Q2, 0)
        e2max = E2(E1, Q2, np.pi)
        e2    = np.append(np.flip(e2min), e2max)
        q2pp  = np.append(np.flip(q2p), q2p)
        plt.plot(E1, q2p,  color='k', ls=ls, label='$\mathrm{1^{st}}$ proton, $Q_2$ = %s MeV' % Q2str)
        plt.fill_betweenx(q2p, e2min, e2max, facecolor='lightgrey', edgecolor='k', hatch=hatch,
                         label='$\mathrm{2^{nd}}$ proton, $Q_2$ = %s MeV' % Q2str, zorder=10)
        #plt.plot(e2, q2pp, color='grey', ls=ls, label='$\mathrm{2^{nd}}$ proton, $Q_2$ = %1.1f MeV' % Q2)
    plt.xlim(-0.3, np.max(E1) + 0.3)
    plt.ylim(-0.3, ymax + 0.3)
    plt.xlabel("$E_i$ (MeV)")
    plt.ylabel("$Q_{\mathrm{2p}}$ (MeV)")
    plt.legend(loc='lower right')
    #plt.legend(bbox_to_anchor=(1., 1.))
    plt.savefig("Q2p_vs_Ei-%s_iterations.pdf" % str(i+1), bbox_inches='tight')
    #plt.savefig("Q2p_vs_Ei-%s_iterations.png" % str(i+1), bbox_inches='tight', dpi=300)
    plt.show()

In [ ]:
al22files = []
seqs = [[30, 35], [40, 40], [45, 52], [55, 55], [57, 59], [61, 61], [63, 74]]
for i in range(len(seqs)):
    for j in range(seqs[i][0], seqs[i][1] + 1):
        al22files.append(proot + "/data/b2p/" + str(j).zfill(3) + "_*mlio.root:a")

In [ ]:
E = Q2p = Theta = np.array([])
for batch in uproot.iterate(al22files, expressions=["E", "Q2p", "Theta"]):
    E = np.append(E, ak.flatten(batch.E))
    Q2p = np.append(Q2p, batch.Q2p)
    Theta = np.append(Theta, batch.Theta)
E = np.array(E[~np.isnan(E)])

In [ ]:
len(E)

In [ ]:
def y1(x, E1):
    return E1 + 0*x
def y2(x, E1, E2): # E1 may be fixed
    M = mp/mNa
    return E2 - 2*M*np.sqrt(E1*E2)*x + E1*M**2

In [ ]:
Q2ps = [[3430, 3480], [4320, 4510]]
Egs  = [0, 1634]

# first entry in E1s is dummy value
E1s  = [
           [
               [2030, 2100]#, [1230, 1290]
           ],
           [   [1810, 1860], [2670, 2700], [2210, 2240], [1330, 1360], 
               #[2430, 2460], [2560, 2590], [2350, 2370]
           ]
       ]
#E1s  = [[1810, 1860], [1330, 1380], [2630, 2760], [3200, 3440], [2350, 2510]]
#E1s = []
#for el in np.arange(500, 4000+100, 100):
#    E1s.append([el, el+100])

X = np.linspace(-1.1, 1.1, 100)

txa = 0.15
tya = 0.84
txb = txa
tyb = 0.3

for i in range(len(Q2ps)):
    used_indices = np.array([]) # to note current indices for removal at end of each E1 iteration
    Q2p_low = Q2ps[i][0]
    Q2p_upp = Q2ps[i][1]
    Q2p_mean = Q2p_low + (Q2p_upp - Q2p_low)/2
    
    prepare_two_column_figure(fig_scale=1.5)
    plt.hist(Q2p, bins=np.arange(800, 8500, 5), 
             histtype='stepfilled', facecolor='k', edgecolor='k', lw=0.4)
    plt.hist(Q2p[(Q2p_low <= Q2p) & (Q2p <= Q2p_upp)], bins=np.arange(800, 8500, 10), 
             histtype='stepfilled', facecolor='r', edgecolor='r', zorder=10)
    plt.savefig('%d___.png' % Q2p_mean, bbox_inches='tight')
    plt.show()
    
    first_round = True
    for j in range(len(E1s[i])+1):
        E1 = E2 = CosTheta = np.array([]) # arrays for events of interest
        E1R = E2R = CosThetaR = np.array([]) # arrays for residual events
        E1_low  = E1s[i][j-1][0]
        E1_upp  = E1s[i][j-1][1]
        print(E1_low, E1_upp)
        
        E1_mean  = E1_low + (E1_upp  - E1_low)/2
        Q1_mean  = E1_mean*(mNa + mp)/mNa
        
        if first_round:
            for k in range(len(Q2p)):
                if Q2p_low <= Q2p[k] and Q2p[k] <= Q2p_upp:
                    E1R = np.append(E1R, E[2*k])
                    E2R = np.append(E2R, E[2*k+1])
                    CosThetaR = np.append(CosThetaR, np.cos(np.deg2rad(Theta[k])))
        else:
            for k in range(len(Q2p)):
                if k in used_indices:
                    continue
                if Q2p_low <= Q2p[k] and Q2p[k] <= Q2p_upp:
                    if Q1_mean > Q2p_mean/2:
                        E1_above = True
                        E1tmp = max(E[2*k], E[2*k+1])
                        E2tmp = min(E[2*k], E[2*k+1])
                    else:
                        E1_above = False
                        E1tmp = min(E[2*k], E[2*k+1])
                        E2tmp = max(E[2*k], E[2*k+1])
                    if E1_low <= E1tmp and E1tmp <= E1_upp:
                        E1 = np.append(E1, E1tmp)
                        E2 = np.append(E2, E2tmp)
                        CosTheta = np.append(CosTheta, np.cos(np.deg2rad(Theta[k])))

                        used_indices = np.append(used_indices, k)
                    else:
                        E1R = np.append(E1R, E1tmp)
                        E2R = np.append(E2R, E2tmp)
                        CosThetaR = np.append(CosThetaR, np.cos(np.deg2rad(Theta[k])))

            if len(E1) == 0:
                print("SKIP", E1s[j-1])
                continue

        E_min = np.min(np.concatenate([E1, E2, E1R, E2R]))
        E_max = np.max(np.concatenate([E1, E2, E1R, E2R]))

        if not first_round:
            p1, c1 = curve_fit(y1, CosTheta, E1, p0=E1_mean)
            p2, c2 = curve_fit(y2, CosTheta, E2, p0=[p1[0], Q2p_mean - Q1_mean], bounds=([p1[0], 0], [p1[0]+1e-12, Q2p_mean]))
        
        fig, axes = prepare_two_column_figure(height_scale=1.25, fig_scale=1.5, nrows=2, ncols=2,
                                             gridspec_kw={'hspace': 0.05, 'height_ratios': [4, 1], 
                                                          'wspace': 0.08, 'width_ratios': [5, 1]})
        
        plt.sca(axes[0,0])
        if not first_round:
            plt.plot(CosTheta, E1, 'ko', ms=6, label='$\mathrm{E_{1}}$')
            plt.plot(CosTheta, E2, 'ko', ms=6, markerfacecolor='w', label='$\mathrm{E_{2}}$')
            plt.plot(X, y1(X, *p1), 'k-', label='Fit to $\mathrm{E_{1}}$')
            plt.plot(X, y2(X, *p2), 'k--', label='Fit to $\mathrm{E_{2}}$')
            plt.plot(CosThetaR, E1R, 'k,', label='Residual data')
            plt.plot(CosThetaR, E2R, 'k,')
        else:
            plt.plot(CosThetaR, E1R, 'k.', label='Data')
            plt.plot(CosThetaR, E2R, 'k.')
        plt.ylabel('$E_{i}$ (keV)')
        plt.ylim(0.9*E_min, 1.1*E_max)
        if not first_round:
            plt.legend(title="Gates: $Q_{\mathrm{2p}}=%d..%d$ keV; $E_{1}=%d..%d$ keV"
                       % (round(Q2p_low), round(Q2p_upp), round(E1_low), round(E1_upp)), ncols=3, 
                       fontsize='x-small', title_fontsize='x-small', loc='upper left', bbox_to_anchor=(0.02, 0.9))
        if not first_round:
            if E1_above:
                X1 = txa
                Y1 = tya
                X2 = txb
                Y2 = tyb
            else:
                X1 = txb
                Y1 = tyb
                X2 = txa
                Y2 = tya
            E1_est = p1[0]
            Q1_est = E1_est*(mNa + mp)/mNa
            E2_est = p2[1]
            Q2_est = E2_est*(mNe + mp)/mNe
            E2_min = y2(1, E1_est, E2_est)
            E2_max = y2(-1, E1_est, E2_est)
            fig.text(X1, Y1, "Fit results: $E_{1}\sim%d$ keV; $Q_{1}\sim%d$ keV; $E_{\mathrm{ex}}^{^{22}\mathrm{Mg}}\sim%d$ keV"
                     % (round(E1_est), round(Q1_est), round(S2p + Egs[i] + Q2_est + Q1_est)),
                    bbox=dict(boxstyle="round", ec='k', fc='w',alpha=0.5,), fontsize='x-small')
            fig.text(X2, Y2, "Fit results: $E_{2}\sim%d\,..%d$ keV; $Q_{2}\sim%d$ keV; $E_{\mathrm{ex}}^{^{21}\mathrm{Na}}\sim%d$ keV" 
                     % (round(E2_min), round(E2_max), round(Q2_est), round(S2p + Egs[i] + Q2_est)),
                    bbox=dict(boxstyle="round", ec='k', fc='w',alpha=0.5,), fontsize='x-small')
        plt.xlim(-1.2, 1.2)
        plt.gca().invert_xaxis()
        plt.xlabel('$\cos{\Theta_{\mathrm{2p}}}$')
#         plt.tick_params('x', labelbottom=False)
        
        
        axes[1,0].remove()
#         plt.sca(axes[1,0])
#         if not first_round:
#             plt.plot(CosTheta, E1 - y1(CosTheta, *p1), 'ko', label='$\mathrm{E_{1}}$')
#             plt.plot(CosTheta, E2 - y2(CosTheta, *p2), 'ko', markerfacecolor='w', label='$\mathrm{E_{1}}$')
#             low, high = plt.ylim()
#             bound = max(abs(low), abs(high))
#             plt.ylim(-1.1*bound, 1.1*bound)
#         plt.plot([-1.3, 1.3], [0, 0], 'k--', lw=0.5)
#         plt.xlim(-1.2, 1.2)
#         plt.xlabel('$\cos{\Theta_{\mathrm{2p}}}$')
#         plt.ylabel('$E_{i}^{\mathrm{data}} - E_{i}^{\mathrm{fit}}$ (keV)')
        
        plt.sca(axes[0,1])
        if not first_round:
            plt.hist(E1, bins=np.arange(E_min, E_max, 10), 
                     edgecolor='k', facecolor='k', histtype='stepfilled', orientation='horizontal', 
                     label='$E_{1}$', zorder=10)
            plt.hist(E2, bins=np.arange(E_min, E_max, 10), 
                     edgecolor='k', facecolor='darkgrey', histtype='stepfilled', orientation='horizontal', 
                     label='$E_{2}$', zorder=11)
            plt.hist(np.append(E1R, E2R), bins=np.arange(E_min, E_max, 10), 
                     color='k', histtype='step', orientation='horizontal', 
                     label='Residual')
            plt.legend(fontsize='xx-small')
        else:
            plt.hist(np.append(E1R, E2R), bins=np.arange(E_min, E_max, 10), 
                     color='k', histtype='step', orientation='horizontal')
        plt.ylim(0.9*E_min, 1.1*E_max)
        plt.xlabel('Counts / 10 keV')
        axes[0,1].yaxis.tick_right()
        
        axes[1,1].remove()
        
        if first_round:
            plt.savefig('%d__initial.png' % Q2p_mean, bbox_inches='tight')
        else:
            plt.savefig('%d_%d.png' % (Q2p_mean, E1_mean), bbox_inches='tight')
        plt.show()
        first_round = False
    
    E1R = E2R = CosThetaR = np.array([]) # arrays for residual events
    for k in range(len(Q2p)):
        if k in used_indices:
            continue
        if Q2p_low <= Q2p[k] and Q2p[k] <= Q2p_upp:
            E1R = np.append(E1R, E[2*k])
            E2R = np.append(E2R, E[2*k+1])
            CosThetaR = np.append(CosThetaR, np.cos(np.deg2rad(Theta[k])))
    fig, axes = prepare_two_column_figure(height_scale=1.25, fig_scale=1.5, nrows=2, ncols=2,
                                             gridspec_kw={'hspace': 0.05, 'height_ratios': [4, 1], 
                                                          'wspace': 0.08, 'width_ratios': [5, 1]})    
    plt.sca(axes[0,0])
    plt.plot(CosThetaR, E1R, 'k.', label='Data')
    plt.plot(CosThetaR, E2R, 'k.')
    plt.ylabel('$E_{i}$ (keV)')
    plt.ylim(0.9*E_min, 1.1*E_max)
    plt.xlim(-1.2, 1.2)
    plt.gca().invert_xaxis()
    plt.tick_params('x', labelbottom=False)
    plt.sca(axes[1,0])
    plt.plot([-1.3, 1.3], [0, 0], 'k--', lw=0.5)
    plt.xlim(-1.2, 1.2)
    plt.xlabel('$\cos{\Theta_{\mathrm{2p}}}$')
    plt.ylabel('$E_{i}^{\mathrm{data}} - E_{i}^{\mathrm{fit}}$ (keV)')
    plt.sca(axes[0,1])
    plt.hist(np.append(E1R, E2R), bins=np.arange(E_min, E_max, 10), 
                 color='k', histtype='step', orientation='horizontal')
    plt.ylim(0.9*E_min, 1.1*E_max)
    plt.xlabel('Counts / 10 keV')
    axes[0,1].yaxis.tick_right()
    axes[1,1].remove()
    plt.savefig('%d_remainder.png' % Q2p_mean, bbox_inches='tight')
    plt.show()